In [1]:
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from typing import Annotated

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
#from langchain_experimental.utilities import PythonREPL
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent
from langgraph.graph import MessagesState

from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages
from langchain_core.messages.ai import AIMessage

In [5]:
class State(MessagesState):
    messages: Annotated[list[AnyMessage], add_messages]

In [6]:
@tool
def resume_screener(resume:str,skill:str):
    """This tool checks andd validates if the resume and the skills passed are in cync or not and returns its response """
    
    llm = ChatOpenAI()
    system = """You are an expert in screenring the resume and selecting OR rejecting the resume based on the skill we require.
                use your atmost intelligence and decide if you want to select the resume OR reject the resume by going through the resume clearly and comparing it will skills we want
                Here are some sample keywords you can check in resume.
                Data Engineer, Big Data, AWS, Azure, SQL Python -- maps to Data Engineer skill
                Data Sceince, ML , Deep Learning, DL, sklearn -- maps to Data Sceince skill
                """
    human_str =  """
                 Here is the resume 
                 ---------
                 {resume}

                 Please check if it maps to the follwoing skills : {skill}
                 """
    route_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", human_str),
        ]
    )

    question_router = route_prompt | llm #structured_llm_router
    response = question_router.invoke({"resume":resume,"skill":skill})
    return {"messages": [response]}

In [ ]:
@tool
def hr_filter(resume:str):
    """This tool checks and validates Abilities, Competencies, Strengths, Capabilities, and Expertise in Resume """

    llm = ChatOpenAI()
    system = """You are a HR specialist tasked with reviewing resumes to evaluate the candidate's suitability for HR-related roles, Please dont be very tough guy, evaluate easy and if resume satisfies the basic HR skills also, you should pass it.
                Focus solely on assessing the candidate's abilities, competencies, strengths, capabilities, and expertise in core HR functions,
                such as talent management, recruitment, employee relations, performance evaluation, training and development, and compliance
                with HR policies. Avoid analyzing technical or non-HR skills and respond with a crisp message if you are satisfied with the candidate on the skills you have evaluated.
                """
    human_str =  """
                 Here is the resume 
                 ---------
                 {resume}
                 """
    route_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", human_str),
        ]
    )

    question_router = route_prompt | llm
    response = question_router.invoke({"resume":resume})
    return {"messages": [response]}

In [ ]:
@tool
def resume_summarize():
    """Summarizes the thoughts from previous agents."""
    # if "messages" not in state or not state["messages"]:
    #     raise ValueError("State must include a 'messages' key with content.")

    llm = ChatOpenAI()
    system = """
        You are a resume summarizer. Summarize feedback from the previous agents.
        Provide one of these decisions: human_review_needed, rejected, selected.
    """

    # Gather all messages from previous agents
    agent_messages = "\n\n".join(
        msg.content for msg in state['messages']
    )

    human_str = """
    Summarize the feedback from the agents based on the messages below:
    ------
    {agent_messages}
    ------
    Provide one of these decisions: human_review_needed, rejected, selected.
    """

    route_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", human_str),
        ]
    )

    question_router = route_prompt | llm
    response = question_router.invoke({"agent_messages": agent_messages})

    # Append the AI message to state
    state["messages"].append(AIMessage(content=response.content))

    return state

In [ ]:
def make_system_prompt(suffix: str) -> str:
    return (
        "You are a helpful AI assistant, collaborating with other assistants."
        " Use the provided tools to progress towards answering the question."
        " If you are unable to fully answer, that's OK, another assistant with different tools "
        " will help where you left off. Execute what you can to make progress."
        " If you or any of the other assistants have the final answer or deliverable,"
        " prefix your response with FINAL ANSWER so the team knows to stop."
        f"\n{suffix}"
    )

In [ ]:
llm = ChatOpenAI()

In [ ]:
resume_screener_agent = create_react_agent(
    llm,
    tools = [resume_screener],
    state_modifier=make_system_prompt(
        "You can filter resumes based on the skill and return if we can select OR reject the resume. select it if resume suits the skillset, reject if resume doesn't suit the skillset" 
    )
)

In [ ]:
def resume_screener_node(state: State) -> State:
    result = resume_screener_agent.invoke(state)
    state["messages"].extend(result["messages"])  # Append summarizer messages
    return state

In [ ]:
hr_filter_agent = create_react_agent(
    llm,
    [hr_filter],
    state_modifier=make_system_prompt(
        "you work with resume screener agent and after only if resume screener agent select , then you will validate the HR skills in the resume"
    ),
)

In [ ]:
def hr_node(state: State) -> State:
    result = hr_filter_agent.invoke(state)
    state["messages"].extend(result["messages"])  # Append summarizer messages
    return state

In [ ]:
resume_summarize_agent = create_react_agent(
    llm,
    [resume_summarize],
    state_modifier=make_system_prompt(
        "you work with resume screener agent, hr agent and just summarises what those 2 agents feel about the resume "
    ),
)

In [ ]:
def resume_summarize_node(state: State) -> State:
    result = resume_summarize_agent.invoke(state)
    state["messages"].extend(result["messages"]) 
    return state